In [1]:
!XLA_FLAGS=--xla_gpu_cuda_data_dir=/cm/shared/sw/pkg/devel/cuda/10.1.243_418.87.00

In [2]:
import numpy as onp

import jax
print("jax version: ", jax.__version__)
import jax.experimental.optimizers as optimizers
import jax.experimental.stax as stax
import jax.numpy as np
from jax import jit
import matplotlib.pyplot as plt
import matplotlib as mpl

jax version:  0.2.10


In [3]:
r"""
A module defining some "nicer" fourier transform functions.
We define only two functions -- an arbitrary-dimension forward transform, and its inverse. In each case, the transform
is designed to replicate the continuous transform. That is, the transform is volume-normalised and obeys correct
Fourier conventions.
The actual FFT backend is provided by ``pyFFTW`` if it is installed, which provides a significant speedup, and
multi-threading.
Conveniently, we allow for arbitrary Fourier convention, according to the scheme in
http://mathworld.wolfram.com/FourierTransform.html. That is, we define the forward and inverse *n*-dimensional
transforms respectively as
.. math:: F(k) = \sqrt{\frac{|b|}{(2\pi)^{1-a}}}^n \int f(r) e^{-i b\mathbf{k}\cdot\mathbf{r}} d^n\mathbf{r}
and
.. math:: f(r) = \sqrt{\frac{|b|}{(2\pi)^{1+a}}}^n \int F(k) e^{+i b\mathbf{k}\cdot\mathbf{r}} d^n \mathbf{k}.
In both transforms, the corresponding co-ordinates are returned so a completely consistent transform is simple to get.
This makes switching from standard frequency to angular frequency very simple.
We note that currently, only positive values for b are implemented (in fact, using negative b is consistent, but
one must be careful that the frequencies returned are descending, rather than ascending).
"""
import jax, warnings

__all__ = ['fft', 'ifft', 'fftfreq', 'fftshift', 'ifftshift']


HAVE_FFTW = False
from jax.numpy.fft import fftn, ifftn, ifftshift as _ifftshift, fftshift as _fftshift, fftfreq as _fftfreq

# To avoid MKL-related bugs, numpy needs to be imported after pyfftw: see https://github.com/pyFFTW/pyFFTW/issues/40
import numpy as np


def fft(X, L=None, Lk=None, a=0, b=2 * np.pi, left_edge=None, axes=None, ret_cubegrid=False):
    r"""
    Arbitrary-dimension nice Fourier Transform.
    This function wraps numpy's ``fftn`` and applies some nice properties. Notably, the returned fourier transform
    is equivalent to what would be expected from a continuous Fourier Transform (including normalisations etc.). In
    addition, arbitrary conventions are supported (see :mod:`powerbox.dft` for details).
    Default parameters have the same normalising conventions as ``numpy.fft.fftn``.
    The output object always has the zero in the centre, with monotonically increasing spectral arguments.
    Parameters
    ----------
    X : array
        An array with arbitrary dimensions defining the field to be transformed. Should correspond exactly
        to the continuous function for which it is an analogue. A lower-dimensional transform can be specified by using
        the ``axes`` argument.
    L : float or array-like, optional
        The length of the box which defines ``X``. If a scalar, each transformed dimension in ``X`` is assumed to have
        the same length. If array-like, must be of the same length as the number of transformed dimensions. The default
        returns the un-normalised DFT (same as numpy).
    Lk : float or array-like, optional
        The length of the fourier-space box which defines the dual of ``X``. Only one of L/Lk needs to be provided. If
        provided, L takes precedence. If a scalar, each transformed dimension in ``X`` is assumed to have
        the same length. If array-like, must be of the same length as the number of transformed dimensions.
    a,b : float, optional
        These define the Fourier convention used. See :mod:`powerbox.dft` for details. The defaults return the standard DFT
        as defined in :mod:`numpy.fft`.
    left_edge : float or array-like, optional
        The co-ordinate at the left-edge for each dimension that is being transformed. By default, sets the left
        edge to -L/2, so that the input is centred before transforming (i.e. equivalent to ``fftshift(fft(fftshift(X)))``)
    axes : sequence of ints, optional
        The axes to take the transform over. The default is to use all axes for the transform.
    ret_cubegrid : bool, optional
        Whether to return the entire grid of frequency magnitudes.
    Returns
    -------
    ft : array
        The DFT of X, normalised to be consistent with the continuous transform.
    freq : list of arrays
        The frequencies in each dimension, consistent with the Fourier conventions specified.
    grid : array
        Only returned if ``ret_cubegrid`` is ``True``. An array with shape given by ``axes`` specifying the magnitude
        of the frequencies at each point of the fourier transform.
    """

    if not HAVE_FFTW:
        warnings.warn("You do not have pyFFTW installed. Installing it should give some speed increase.")

    if axes is None:
        axes = list(range(len(X.shape)))

    N = np.array([X.shape[axis] for axis in axes])

    # Get the box volume if given the fourier-space box volume
    if L is None and Lk is None:
        L = N
    elif L is not None:  # give precedence to L
        if np.isscalar(L):
            L = L * np.ones(len(axes))
    elif Lk is not None:
        if np.isscalar(Lk):
            Lk = Lk * np.ones(len(axes))
        L = N * 2 * np.pi / (Lk * b)  # Take account of the fourier convention.

    left_edge = _set_left_edge(left_edge, axes, L)

    V = float(np.product(L))  # Volume of box
    Vx = V / np.product(N)  # Volume of cell

    ft = Vx * fftshift(fftn(X, axes=axes), axes=axes) * np.sqrt(np.abs(b) / (2 * np.pi) ** (1 - a)) ** len(axes)

    dx = np.array([float(l) / float(n) for l, n in zip(L, N)])

    freq = np.array([fftfreq(n, d=d, b=b) for n, d in zip(N, dx)])

    # Adjust phases of the result to align with the left edge properly.
    ft = _adjust_phase(ft, left_edge, freq, axes, b)
    return _retfunc(ft, freq, axes, ret_cubegrid)


def ifft(X, Lk=None, L=None, a=0, b=2 * np.pi, axes=None, left_edge=None, ret_cubegrid=False):
    r"""
    Arbitrary-dimension nice inverse Fourier Transform.
    This function wraps numpy's ``ifftn`` and applies some nice properties. Notably, the returned fourier transform
    is equivalent to what would be expected from a continuous inverse Fourier Transform (including normalisations etc.).
    In addition, arbitrary conventions are supported (see :mod:`powerbox.dft` for details).
    Default parameters have the same normalising conventions as ``numpy.fft.ifftn``.
    Parameters
    ----------
    X : array
        An array with arbitrary dimensions defining the field to be transformed. Should correspond exactly
        to the continuous function for which it is an analogue. A lower-dimensional transform can be specified by using
        the ``axes`` argument. Note that if using a non-periodic function, the co-ordinates should be monotonically
        increasing.
    Lk : float or array-like, optional
        The length of the box which defines ``X``. If a scalar, each transformed dimension in ``X`` is assumed to have
        the same length. If array-like, must be of the same length as the number of transformed dimensions. The default
        returns the un-normalised DFT (the same as numpy).
    L : float or array-like, optional
        The length of the real-space box, defining the dual of ``X``. Only one of Lk/L needs to be passed. If L is
        passed, it is used. If a scalar, each transformed dimension in ``X`` is assumed to have
        the same length. If array-like, must be of the same length as the number of transformed dimensions. The default
        of ``Lk=1`` returns the un-normalised DFT.
    a,b : float, optional
        These define the Fourier convention used. See :mod:`powerbox.dft` for details. The defaults return the standard DFT
        as defined in :mod:`numpy.fft`.
    axes : sequence of ints, optional
        The axes to take the transform over. The default is to use all axes for the transform.
    left_edge : float or array-like, optional
        The co-ordinate at the left-edge (in k-space) for each dimension that is being transformed. By default, sets the
        left edge to -Lk/2, equivalent to the standard numpy ifft. This affects only the phases of the result.
    ret_cubegrid : bool, optional
        Whether to return the entire grid of real-space co-ordinate magnitudes.
    Returns
    -------
    ft : array
        The IDFT of X, normalised to be consistent with the continuous transform.
    freq : list of arrays
        The real-space co-ordinate grid in each dimension, consistent with the Fourier conventions specified.
    grid : array
        Only returned if ``ret_cubegrid`` is ``True``. An array with shape given by ``axes`` specifying the magnitude
        of the real-space co-ordinates at each point of the inverse fourier transform.
    """

    if not HAVE_FFTW:
        warnings.warn("You do not have pyFFTW installed. Installing it should give some speed increase.")

    if axes is None:
        axes = list(range(len(X.shape)))

    N = np.array([X.shape[axis] for axis in axes])

    # Get the box volume if given the real-space box volume
    if Lk is None and L is None:
        Lk = 1
    elif L is not None:
        if np.isscalar(L):
            L = np.array([L] * len(axes))

        dx = L / N
        Lk = 2 * np.pi / (dx * b)

    elif np.isscalar(Lk):
        Lk = [Lk] * len(axes)

    Lk = np.array(Lk)
    left_edge = _set_left_edge(left_edge, axes, Lk)

    V = np.product(Lk)
    dk = np.array([float(lk) / float(n) for lk, n in zip(Lk, N)])

    ft = V * ifftn(X, axes=axes) * np.sqrt(np.abs(b) / (2 * np.pi) ** (1 + a)) ** len(axes)
    ft = ifftshift(ft, axes=axes)

    freq = np.array([fftfreq(n, d=d, b=b) for n, d in zip(N, dk)])

    ft = _adjust_phase(ft, left_edge, freq, axes, -b)
    return _retfunc(ft, freq, axes, ret_cubegrid)


def _adjust_phase(ft, left_edge, freq, axes, b):
    for i, (l, f) in enumerate(zip(left_edge, freq)):
        xp = np.exp(-b * 1j * f * l)
        obj = tuple([None] * axes[i]) + (slice(None, None, None),) + tuple([None] * (ft.ndim - axes[i] - 1))
        ft *= xp[obj]
    return ft


def _set_left_edge(left_edge, axes, L):
    if left_edge is None:
        left_edge = [-l/2. for l in L]
    else:
        if np.isscalar(left_edge):
            left_edge = [left_edge] * len(axes)
        else:
            assert len(left_edge) == len(axes)

    return left_edge


def _retfunc(ft, freq, axes, ret_cubegrid):
    if not ret_cubegrid:
        return ft, freq
    else:
        grid = freq[0] ** 2
        for i in range(1, len(axes)):
            grid = np.add.outer(grid, freq[i] ** 2)

        return ft, freq, np.sqrt(grid)


def fftshift(x, *args, **kwargs):
    """
    The same as numpy's fftshift, except that it preserves units (if Astropy quantities are used)
    
    All extra arguments are passed directly to numpy's `fftshift`.
    """
    out = _fftshift(x, *args, **kwargs)

    if hasattr(x, "unit"):
        return out * x.unit
    else:
        return out


def ifftshift(x, *args, **kwargs):
    """
    The same as numpy's ifftshift, except that it preserves units (if Astropy quantities are used)
    All extra arguments are passed directly to numpy's `ifftshift`.
    """
    out = _ifftshift(x, *args, **kwargs)

    if hasattr(x, "unit"):
        return out * x.unit
    else:
        return out


def fftfreq(N, d=1.0, b=2 * np.pi):
    """
    Return the fourier frequencies for a box with N cells, using general Fourier convention.
    Parameters
    ----------
    N : int
        The number of grid cells
    d : float, optional
        The interval between cells
    b : float, optional
        The fourier-convention of the frequency component (see :mod:`powerbox.dft` for details).
    Returns
    -------
    freq : array
        The N symmetric frequency components of the Fourier transform. Always centred at 0.
    """
    return fftshift(_fftfreq(N, d=d)) * (2 * np.pi / b)

In [4]:
def _magnitude_grid(x, dim=None):
    if dim is not None:
        return np.sqrt(np.sum(np.meshgrid(*([x ** 2] * dim)), axis=0))
    else:
        return np.sqrt(np.sum(np.meshgrid(*([X ** 2 for X in x])), axis=0))

In [ ]:
"""
A module defining two classes which can create arbitrary-dimensional fields with given power spectra. One such function
produces *Gaussian* fields, and the other *LogNormal* fields.
In principle, these may be extended to other 1-point density distributions by subclassing :class:`PowerBox` and
over-writing the same methods as are over-written in :class:`LogNormalPowerBox`.
"""

import jax.numpy as np
#from . import dft
from .tools import _magnitude_grid

try:
    from multiprocessing import cpu_count

    THREADS = cpu_count()

    from pyfftw import empty_aligned as empty

    HAVE_FFTW = True
except ImportError:
    empty = np.empty
    HAVE_FFTW = False


# TODO: add hankel-transform version of LogNormal


def _make_hermitian(mag, pha):
    r"""
    Take random arrays and convert them to a complex hermitian array.
    Note that this assumes that mag is distributed normally.
    Parameters
    ----------
    mag : array
        Normally-distributed magnitudes of the complex vector.
    pha : array
        Uniformly distributed phases of the complex vector
    Returns
    -------
    kspace : array
        A complex hermitian array with normally distributed amplitudes.
    """
    revidx = (slice(None, None, -1),) * len(mag.shape)
    mag = (mag + mag[revidx]) / np.sqrt(2)
    pha = (pha - pha[revidx]) / 2 + np.pi
    return mag * (np.cos(pha) + 1j * np.sin(pha))


class myPowerBox(object):
    r"""
    Calculate real- and fourier-space Gaussian fields generated with a given power spectrum.
    Parameters
    ----------
    N : int
        Number of grid-points on a side for the resulting box (equivalently, number of wavenumbers to use).
    pk : callable
        A callable of a single (vector) variable `k`, which is the isotropic power spectrum. The relationship of the
        `k` of which this is a function to the real-space co-ordinates, `x`, is determined by the parameters ``a,b``.
    dim : int, default 2
        Number of dimensions of resulting box.
    boxlength : float, default 1.0
        Length of the final signal on a side. This may have arbitrary units, so long as `pk` is a function of a
        variable which has the inverse units.
    ensure_physical : bool, optional
        Interpreting the power spectrum as a spectrum of density fluctuations, the minimum physical value of the
        real-space field, :meth:`delta_x`, is -1. With ``ensure_physical`` set to ``True``, :meth:`delta_x` is
        clipped to return values >-1. If this is happening a lot, consider using :class:`LogNormalPowerBox`.
    a,b : float, optional
        These define the Fourier convention used. See :mod:`powerbox.dft` for details. The defaults define the standard
        usage in *cosmology* (for example, as defined in Cosmological Physics, Peacock, 1999, pg. 496.). Standard
        numerical usage (eg. numpy) is (a,b) = (0,2pi).
    vol_normalised_power : bool, optional
        Whether the input power spectrum, ``pk``, is volume-weighted. Default True because of standard cosmological
        usage.
    seed: int, optional
        A random seed to define the initial conditions. If not set, it will remain random, and each call to eg.
        :meth:`delta_x()` will produce a *different* realisation.
    Notes
    -----
    A number of conventions need to be listed.
    The conventions of using `x` for "real-space" and `k` for "fourier space" arise from cosmology, but this does
    not affect anything -- `x` could just as well stand for "time domain" and `k` for "frequency domain".
    The important convention is the relationship between `x` and `k`, or in other words, whether `k` is interpreted
    as an angular frequency or ordinary frequency. By default, because of cosmological conventions, `k` is an
    angular frequency, so that the fourier transform integrand is delta_k*exp(-ikx). The conventions can be changed
    arbitrarily by setting the ``a,b`` parameters (see :mod:`powerbox.dft` for details).
    The primary quantity of interest is :meth:`delta_x`, which is a zero-mean Gaussian field with a power spectrum
    equivalent to that which was input. Being zero-mean enables its direct interpretation as an overdensity
    field, and this interpretation is enforced in the :meth:`make_discrete_sample` method.
    .. note:: None of the n-dimensional arrays that are created within the class are stored, due to the inefficiency
              in memory consumption that this would imply. Thus, each large array is created and *returned* by their
              respective method, to be stored/discarded by the user.
    .. warning:: Due to the above note, repeated calls to eg. :meth:`delta_x()` will produce *different* realisations
                 of the real-space field, unless the `seed` parameter is set in the constructor.
    Examples
    --------
    To create a 3-dimensional box of gaussian over-densities, gridded into 100 bins, with cosmological conventions,
    and a power-law power spectrum, simply use
    >>> pb = PowerBox(100,lambda k : 0.1*k**-3., dim=3, boxlength=100.0)
    >>> overdensities = pb.delta_x()
    >>> grid = pb.x
    >>> radii = pb.r
    To create a 2D turbulence structure, with arbitrary units, once can use
    >>> import matplotlib.pyplot as plt
    >>> pb = PowerBox(1000, lambda k : k**-7./5.)
    >>> plt.imshow(pb.delta_x())
    """

    def __init__(self, N, pk, dim=2, boxlength=1.0, ensure_physical=False, a=1., b=1.,
                 vol_normalised_power=True, seed=None):

        self.N = N
        self.dim = dim
        self.boxlength = boxlength
        self.L = boxlength
        self.fourier_a = a
        self.fourier_b = b
        self.vol_normalised_power = vol_normalised_power
        self.V = self.boxlength ** self.dim

        if self.vol_normalised_power:
            self.pk = lambda k: pk(k) / self.V
        else:
            self.pk = pk

        self.ensure_physical = ensure_physical
        self.Ntot = self.N ** self.dim

        self.seed = seed

        if N % 2 == 0:
            self._even = True
        else:
            self._even = False

        self.n = N + 1 if self._even else N

        # Get the grid-size for the final real-space box.
        self.dx = float(boxlength) / N

    def k(self):
        "The entire grid of wavenumber magitudes"
        return _magnitude_grid(self.kvec, self.dim)

    @property
    def kvec(self):
        "The vector of wavenumbers along a side"
        return dft.fftfreq(self.N, d=self.dx, b=self.fourier_b)

    @property
    def r(self):
        "The radial position of every point in the grid"
        return _magnitude_grid(self.x, self.dim)

    @property
    def x(self):
        "The co-ordinates of the grid along a side"
        return np.arange(-self.boxlength / 2, self.boxlength / 2, self.dx)[:self.N]

    def gauss_hermitian(self):
        "A random array which has Gaussian magnitudes and Hermitian symmetry"
        if self.seed:
            np.random.seed(self.seed)

        mag = np.random.normal(0, 1, size=[self.n] * self.dim)
        pha = 2 * np.pi * np.random.uniform(size=[self.n] * self.dim)

        dk = _make_hermitian(mag, pha)

        if self._even:
            cutidx = (slice(None, -1),) * self.dim
            dk = dk[cutidx]

        return dk

    def power_array(self):
        "The Power Spectrum (volume normalised) at `self.k`"
        k = self.k()
        mask = k != 0
        # Re-use the k array to conserve memory
        k[mask] = self.pk(k[mask])
        return k

    def delta_k(self):
        "A realisation of the delta_k, i.e. the gaussianised square root of the power spectrum (i.e. the Fourier co-efficients)"
        p = self.power_array()

        if np.any(p < 0):
            raise ValueError("The power spectrum function has returned negative values.")

        gh = self.gauss_hermitian()
        gh[...] = np.sqrt(p) * gh
        return gh

    def delta_x(self):
        "The realised field in real-space from the input power spectrum"
        # Here we multiply by V because the (inverse) fourier-transform of the (dimensionless) power has
        # units of 1/V and we require a unitless quantity for delta_x.
        dk = empty((self.N,) * self.dim, dtype='complex128')
        dk[...] = self.delta_k()
        dk[...] = self.V * dft.ifft(dk, L=self.boxlength, a=self.fourier_a, b=self.fourier_b)[0]
        dk = np.real(dk)

        if self.ensure_physical:
            np.clip(dk, -1, np.inf, dk)

        return dk

    def create_discrete_sample(self, nbar, randomise_in_cell=True, min_at_zero=False,
                               store_pos=False):
        r"""
        Assuming that the real-space signal represents an over-density with respect to some mean, create a sample
        of tracers of the underlying density distribution.
        Parameters
        ----------
        nbar : float
            Mean tracer density within the box.
        randomise_in_cell : bool, optional
            Whether to randomise the positions of the tracers within the cells, or put them at the grid-points (more
            efficient).
        min_at_zero : bool, optional
            Whether to make the lower corner of the box at the origin, otherwise the centre of the box is at the
            origin.
        store_pos : bool, optional
            Whether to store the sample of tracers as an instance variable `tracer_positions`.
        Returns
        -------
        tracer_positions : float, array_like
            ``(n, d)``-array, with ``n`` the number of tracers and ``d`` the number of dimensions. Each row represents
            a single tracer's co-ordinates.
        """
        dx = self.delta_x()
        dx = (dx + 1) * self.dx ** self.dim * nbar
        n = dx
        self.n_per_cell = np.random.poisson(n)

        # Get all source positions
        args = [self.x] * self.dim
        X = np.meshgrid(*args)

        tracer_positions = np.array([x.flatten() for x in X]).T
        tracer_positions = tracer_positions.repeat(self.n_per_cell.flatten(), axis=0)

        if randomise_in_cell:
            tracer_positions += np.random.uniform(size=(np.sum(self.n_per_cell), self.dim)) * self.dx

        if min_at_zero:
            tracer_positions += self.boxlength / 2.0

        if store_pos:
            self.tracer_positions = tracer_positions

        return tracer_positions


class myLogNormalPowerBox(PowerBox):
    r"""
    Calculate Log-Normal density fields with given power spectra.
    See the documentation of :class:`PowerBox` for a detailed explanation of the arguments, as this class
    has exactly the same arguments.
    This class calculates an (over-)density field of arbitrary dimension given an input isotropic power spectrum. In
    this case, the field has a log-normal distribution of over-densities, always yielding a physically valid field.
    Examples
    --------
    To create a log-normal over-density field:
    >>> from powerbox import LogNormalPowerBox
    >>> lnpb = LogNormalPowerBox(100,lambda k : k**-7./5.,dim=2, boxlength=1.0)
    >>> overdensities = lnpb.delta_x
    >>> grid = lnpb.x
    >>> radii = lnpb.r
    To plot the overdensities:
    >>> import matplotlib.pyplot as plt
    >>> plt.imshow(pb.delta_x)
    Compare the fields from a Gaussian and Lognormal realisation with the same power:
    >>> lnpb = LogNormalPowerBox(300,lambda k : k**-7./5.,dim=2, boxlength=1.0)
    >>> pb = PowerBox(300,lambda k : k**-7./5.,dim=2, boxlength=1.0)
    >>> fig,ax = plt.subplots(2,1,sharex=True,sharey=True,figsize=(12,5))
    >>> ax[0].imshow(lnpb.delta_x,aspect="equal",vmin=-1,vmax=lnpb.delta_x.max())
    >>> ax[1].imshow(pb.delta_x,aspect="equal",vmin=-1,vmax = lnpb.delta_x.max())
    To create and plot a discrete version of the field:
    >>> positions = lnpb.create_discrete_sample(nbar=1000.0, # Number density in terms of boxlength units
    >>>                                         randomise_in_cell=True)
    >>> plt.scatter(positions[:,0],positions[:,1],s=2,alpha=0.5,lw=0)
    """

    def __init__(self, *args, **kwargs):
        super(LogNormalPowerBox, self).__init__(*args, **kwargs)

    def correlation_array(self):
        "The correlation function from the input power, on the grid"
        pa = empty((self.N,) * self.dim)
        pa[...] = self.power_array()
        return self.V * np.real(dft.ifft(pa, L=self.boxlength, a=self.fourier_a, b=self.fourier_b)[0])

    def gaussian_correlation_array(self):
        "The correlation function required for a Gaussian field to produce the input power on a lognormal field"
        return np.log(1 + self.correlation_array())

    def gaussian_power_array(self):
        "The power spectrum required for a Gaussian field to produce the input power on a lognormal field"
        gca = empty((self.N,) * self.dim)
        gca[...] = self.gaussian_correlation_array()
        gpa = np.abs(dft.fft(gca, L=self.boxlength, a=self.fourier_a, b=self.fourier_b))[0]
        gpa[self.k() == 0] = 0
        return gpa

    def delta_k(self):
        """
        A realisation of the delta_k, i.e. the gaussianised square root of the unitless power spectrum
        (i.e. the Fourier co-efficients)
        """
        p = self.gaussian_power_array()
        gh = self.gauss_hermitian()
        gh[...] = np.sqrt(p) * gh
        return gh

    def delta_x(self):
        "The real-space over-density field, from the input power spectrum"
        dk = empty((self.N,) * self.dim, dtype='complex128')
        dk[...] = self.delta_k()
        dk[...] = np.sqrt(self.V) * dft.ifft(dk, L=self.boxlength, a=self.fourier_a, b=self.fourier_b)[0]
        dk = np.real(dk)

        sg = np.var(dk)
        return np.exp(dk - sg / 2) - 1